In [ ]:
import os
import h5py
import numpy as np
import pandas as pd
from synthesizer.conversions import lnu_to_absolute_mag
from unyt import erg, Hz, s  # Ensure unyt package is installed

# Define input and output paths
input_dir = "/disk/xray15/aem2/data/28pams/IllTNG"
output_dir = "/disk/xray15/aem2/data/28pams/IllTNG/LFs"
os.makedirs(output_dir, exist_ok=True)
#snaps available:
#snap_044, z=2.00 = 2
#snap_052, z=1.48 = 1.5
#snap_060, z=1.05 = 1
#snap_086, z=0.10 = 0.1

# Snapshot data for redshift
redshift_values = {
    'snap_044': (2.00, '044'),
    'snap_052': (1.48, '052'),
    'snap_060': (1.05, '060'),
    'snap_086': (0.10, '086')
}

# Co-moving volume parameters
BoxSize = 75.0  # Placeholder, in Mpc/h
little_h = 0.7
massBinLimits = np.arange(-24, -16, 0.5)

def calc_df(abs_mag, volume, massBinLimits):
    hist, _ = np.histogram(abs_mag, bins=massBinLimits)
    hist = np.float64(hist)
    phi = (hist / volume) / (massBinLimits[1] - massBinLimits[0])
    phi_sigma = (np.sqrt(hist) / volume) / (massBinLimits[1] - massBinLimits[0])
    return phi, phi_sigma, hist

# Process each file in the directory
for filename in os.listdir(input_dir):
    if filename.endswith("_photometry.hdf5"):
        file_path = os.path.join(input_dir, filename)

        # Process each snapshot
        for snap, (redshift, snap_code) in redshift_values.items():
            # Define output filename based on input filename and snapshot code
            output_filename = f"LF_{filename.replace('_photometry.hdf5', '')}_{snap_code}.txt"
            output_path = os.path.join(output_dir, output_filename)

            # Calculate co-moving volume for this redshift
            Vphys = (BoxSize / little_h) ** 3
            Vcom = Vphys * ((1 + redshift) ** 3)

            with h5py.File(file_path, 'r') as hdf:
                # Access UV luminosity data for this snapshot
                lum_UV1500 = hdf[f'/{snap}/BC03/photometry/luminosity/intrinsic/UV1500'][:]
                lum_UV1500_unyt = lum_UV1500 * erg / (Hz * s)

                # Convert to absolute magnitudes
                abs_mags = lnu_to_absolute_mag(lum_UV1500_unyt)

                # Calculate luminosity function
                phi, phi_sigma, hist = calc_df(abs_mags, Vcom, massBinLimits)

                # Create DataFrame and save to TXT file
                df = pd.DataFrame({
                    'phi': phi,
                    'phi_sigma': phi_sigma,
                    'hist': hist
                })
                df.to_csv(output_path, index=False, sep='\t')

            print(f"Processed {filename} for {snap}, output saved to {output_path}")


Processed IllustrisTNG_1P_p20_n2_photometry.hdf5 for snap_044, output saved to /disk/xray15/aem2/data/28pams/IllTNG/LFs/LF_IllustrisTNG_1P_p20_n2_044.txt
Processed IllustrisTNG_1P_p20_n2_photometry.hdf5 for snap_052, output saved to /disk/xray15/aem2/data/28pams/IllTNG/LFs/LF_IllustrisTNG_1P_p20_n2_052.txt
Processed IllustrisTNG_1P_p20_n2_photometry.hdf5 for snap_060, output saved to /disk/xray15/aem2/data/28pams/IllTNG/LFs/LF_IllustrisTNG_1P_p20_n2_060.txt
Processed IllustrisTNG_1P_p20_n2_photometry.hdf5 for snap_086, output saved to /disk/xray15/aem2/data/28pams/IllTNG/LFs/LF_IllustrisTNG_1P_p20_n2_086.txt
Processed IllustrisTNG_1P_p17_n1_photometry.hdf5 for snap_044, output saved to /disk/xray15/aem2/data/28pams/IllTNG/LFs/LF_IllustrisTNG_1P_p17_n1_044.txt
Processed IllustrisTNG_1P_p17_n1_photometry.hdf5 for snap_052, output saved to /disk/xray15/aem2/data/28pams/IllTNG/LFs/LF_IllustrisTNG_1P_p17_n1_052.txt
Processed IllustrisTNG_1P_p17_n1_photometry.hdf5 for snap_060, output saved 